In [3]:
!pip install spacy datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [18]:
from datasets import load_dataset
from spacy.tokens import DocBin
import spacy
import json

In [19]:
dataset = load_dataset('eriktks/conll2003')
dataset['train'].to_json("train.json", orient='records', lines=True)
dataset['validation'].to_json("dev.json", orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

988659

In [20]:
def json_to_spacy(json_file, output_file, label_list):
    nlp = spacy.blank("en")
    doc_bin = DocBin()

    with open(json_file, "r") as f:
        for line in f:
            data = json.loads(line)
            tokens = data["tokens"]
            tags = data["ner_tags"]
            text = " ".join(tokens)
            doc = nlp.make_doc(text)

            ents = []
            start = 0
            for token, tag_id in zip(tokens, tags):
                word_start = doc.text.find(token, start)
                word_end = word_start + len(token)
                start = word_end
                label = label_list[tag_id]
                if label != "O":
                    span = doc.char_span(word_start, word_end, label=label)
                    if span:
                        ents.append(span)

            doc.ents = ents
            doc_bin.add(doc)

    doc_bin.to_disk(output_file)

In [22]:
label_names = [
    'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG',
    'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'
]
json_to_spacy("train.json", "train.spacy", label_names)
json_to_spacy("dev.json", "dev.spacy", label_names)

In [25]:
!python -m spacy init config config.cfg --lang en --pipeline ner -F

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [29]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id 0


ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.26    0.00    0.00    0.00    0.00
  0     200         88.75   3440.24   58.07   60.14   56.13    0.58
  0     400        223.01   2208.43   73.98   75.40   72.61    0.74
  0     600        235.91   2090.29   78.50   80.11   76.95    0.78
  0     800        306.99   2151.11   82.86   84.10   81.66    0.83
  0    1000        745.92   2315.42   83.83   85.07   82.63    0.84
  1    1200        438.19   2218.10   86.34   87.22   85.47    0.86
  1    1400        483.52   1654.63   85.80   86.26   85.34    0.86
  1    1600        476.01   

In [33]:
nlp = spacy.load("output/model-best")
doc = nlp("Ahmed asked Nasa to visit the Moon.")

print([(ent.text, ent.label_) for ent in doc.ents])

[('Ahmed', 'B-PER'), ('Nasa', 'B-PER'), ('Moon', 'B-LOC')]
